In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
#py文件 导入数据
#数据与图片文件夹已导入
from lr_utils import load_dataset

%matplotlib inline

In [ ]:
#导入数据（cat/non-cat）
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [ ]:
#示例图片
index = 5
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

In [ ]:
#我觉得写算法关键的地方就在于数据的结构，搞清结构后才能顺利进行其他调参
#观察数据的结构
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
#必须修改数据的结构 用于计算 
#拉直 并转制T
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

In [ ]:
#改变值（由于颜色值在0-255之间 所以做一次缩小？）
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

In [ ]:
#非作业代码    可以看出形状没变 值变了
print(train_set_x.shape)
print(train_set_x_flatten.shape)
print(train_set_x==train_set_x_flatten)

In [ ]:
#sigmoid
def sigmoid(z):

    s = 1 / (1 + np.exp(-z))
    
    return s

In [ ]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

In [ ]:
#初始化权重w和偏向b
def initialize_with_zeros(dim):

    w = np.zeros((dim, 1))
    b = 0

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [ ]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

In [ ]:
#正向传递 
def propagate(w, b, X, Y):

    m = X.shape[1]
    
    #进行单元计算 得到单元输出值A
    A = sigmoid(np.dot(w.T, X) + b)            # compute activation
    #计算成本函数 
    cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))         # compute cost
    #更新w，b的梯度
    dw = 1 / m * np.dot(X, (A - Y).T)
    db = 1 / m * np.sum(A - Y)
    #检测dw，db的形状和属性
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    #压缩维度
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
w, b, X, Y = np.array([[1],[2]]), 2, np.array([[1,2],[3,4]]), np.array([[1,0]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

In [ ]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
   
    
    costs = []
    
    for i in range(num_iterations):
        #从上一个方法中获得梯度与价值函数的值
        grads, cost = propagate(w, b, X, Y)
        
        #重新赋值dw db
        dw = grads["dw"]
        db = grads["db"]
        
        #更新w和b 使用学习率
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        #每循环一百次 记录一次cost
        if i % 100 == 0:
            costs.append(cost)
        
        #print_cost为True的话，每百次输出一次
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print(costs)

In [ ]:
#预测函数
def predict(w, b, X):
    #X.shape[1] = 2
    m = X.shape[1]
    #Y_prediction.shape = (1,2)
    Y_prediction = np.zeros((1,m))
    #X.shape[0] = 2 
    #w.shape = (2,1)
    w = w.reshape(X.shape[0], 1)
    #计算单元输出
    A = sigmoid(np.dot(w.T, X) + b)
    
    #进行判断 sigmoid的值以0.5为区分进行预测
    for i in range(A.shape[1]):
        
        if A[0, i] <= 0.5:
            Y_prediction[0, i] = 0
        else:
            Y_prediction[0, i] = 1
    #检测形状
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [ ]:
print ("predictions = " + str(predict(w, b, X)))


In [ ]:
#算法模型方法
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

    
    #初始化参数 形状与X_train相同
    w, b = initialize_with_zeros(X_train.shape[0])

    #计算梯度
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    #w，b赋值
    w = parameters["w"]
    b = parameters["b"]
    
    #进行预测
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)


    #打印 train/test 准确度 
    #np.mean(np.abs(Y_prediction_train - Y_train)求错误率
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    #
    #print(Y_prediction_train)
    #print(Y_train)
    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

In [ ]:
#一个错误事例
index = 10
plt.imshow(test_set_x[:,index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[0,index]) + ", you predicted that it is a \"" + classes[int(d["Y_prediction_test"][0,index])].decode("utf-8") +  "\" picture.")

In [ ]:
# 画出代价函数
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

In [ ]:

learning_rates = [0.01, 0.001, 0.0001]
models = {}
#用不同的学习率去计算模型得到的结果不同
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
    print ('\n' + "-------------------------------------------------------" + '\n')
#画出不同的学习率得到的曲线
for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

In [ ]:
## 测试你自己的照片
my_image = "cat_in_iran.jpg"   # change this to the name of your image file 


# We preprocess the image to fit your algorithm.
fname = "images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((1, num_px*num_px*3)).T
my_predicted_image = predict(d["w"], d["b"], my_image)

plt.imshow(image)
print("y = " + str(np.squeeze(my_predicted_image)) + ", your algorithm predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")